In [9]:
import pandas as pd
import numpy as np
from transformers import pipeline
from transformers import RobertaTokenizer, RobertaModel, AutoModel, AutoTokenizer

In [2]:
all_data = pd.read_csv("vaalit_2019.csv")

In [3]:
print(all_data.info())

# Look at lappi related columns
lappi_columns = [col for col in all_data.columns if col[:6] == 'Lappi.']
print(lappi_columns, len(lappi_columns))

# Print answers to a lappi question. We'll use these to test the encoder.
lappi_question_idx = -1
lappi_data = all_data[lappi_columns[lappi_question_idx]]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2437 entries, 0 to 2436
Columns: 212 entries, vaalipiiri to Lappi. Jäämeren rata pitää rakentaa..1
dtypes: float64(5), int64(1), object(206)
memory usage: 3.9+ MB
None
['Lappi. Lappiin ei saa avata yhtään uutta kaivosta ennen kuin yhtiöiltä aletaan periä kaivosveroa.', 'Lappi. Mahdollisen uuden maakuntahallinnon toimintoja ja työpaikkoja on sijoitettava muuallekin kuin Rovaniemelle.', 'Lappi. Lapin erikoissairaanhoito on keskitettävä kokonaan Rovaniemelle Lapin keskussairaalaan.', 'Lappi. Alkuperäiskansojen oikeuksia koskeva ILO-sopimus pitää ratifioida seuraavalla hallituskaudella. ', 'Lappi. Jäämeren rata pitää rakentaa.', 'Lappi. Lappiin ei saa avata yhtään uutta kaivosta ennen kuin yhtiöiltä aletaan periä kaivosveroa..1', 'Lappi. Mahdollisen uuden maakuntahallinnon toimintoja ja työpaikkoja on sijoitettava muuallekin kuin Rovaniemelle..1', 'Lappi. Lapin erikoissairaanhoito on keskitettävä kokonaan Rovaniemelle Lapin keskussairaalaan

In [5]:
print(lappi_data.iloc[2432])

Nykyinen ratasuunnitelma on todettu tutkimuksissa nykytilanteessa kannattamattomaksi. Lisäksi hanke aiheuttaa ongelmia ympäristölle ja alueen väestölle. Suunnitelmassa on kuitenkin yksi mielenkiintoinen osa. Junaradan jatkaminen Kemijärveltä Pyhätunturin kautta Sodankylään voisi olla toteuttamiskelpoinen ajatus.

Ratayhteys Jäämerelle voidaan saada aikaan toisella tavalla varsin helposti ja selvästi halvemmalla. Päivitetään nykyinen  Laurila-Tornio-Haaparanta ratayhteys. Tämän päivityksen hinta on vain murto-osa nyt suunnitellusta linjauksesta ja sillä saavutettaisiin ratayhteys Jäämeren ja Atlantin lisäksi myös keskeiseen Eurooppaan ja vaikka Välimerelle saakka. 


In [17]:
# Load both turku and roberta models. Turku seems more rigorous and documented.
model_id_turku = 'TurkuNLP/bert-large-finnish-cased-v1'
model_id_roberta = 'Finnish-NLP/roberta-large-finnish'

tokenizer_roberta = RobertaTokenizer.from_pretrained(model_id_roberta)
tokenizer_turku = AutoTokenizer.from_pretrained(model_id_turku)

model_roberta = AutoModel.from_pretrained(model_id_roberta)
model_turku = AutoModel.from_pretrained(model_id_turku)

models = [model_roberta, model_turku]

Some weights of the model checkpoint at Finnish-NLP/roberta-large-finnish were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at Finnish-NLP/roberta-large-finnish and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and i

In [18]:
# To look at model architectures
# models[0]
# models[1]

In [19]:
# Test with a single row on dataset.
text = lappi_data.iloc[2432]
tokenized_input = tokenizer_turku(text, return_tensors='pt')
token_encodings = models[1](**tokenized_input)

print(token_encodings)

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 3.5597, -0.1192, -0.4937,  ...,  0.5554,  0.9808,  0.3700],
         [ 4.0428, -0.5044, -0.6349,  ..., -0.0724, -1.4574,  0.8273],
         [ 3.6685, -0.2831, -1.1789,  ..., -1.1386, -0.1853,  0.3881],
         ...,
         [ 3.0214,  1.7404, -0.3069,  ..., -0.1230, -0.4943,  0.1827],
         [ 3.9432,  1.9230, -0.7434,  ..., -0.5298,  0.7216,  0.0497],
         [-3.4648,  1.1225, -0.6437,  ...,  0.5395, -0.4789,  0.8119]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[ 0.0730, -1.0000,  0.9520,  ..., -0.2425, -0.1296, -0.3292]],
       grad_fn=<TanhBackward0>), hidden_states=None, past_key_values=None, attentions=None, cross_attentions=None)


In [16]:
# Pooled output is just the hidden state of the first token passed through a linear layer.
# Should we to start with just do avg pooling on all tokens and represent that as a node feature representing a single sentence?

# print(token_encodings['input_ids'].shape) # B, T
print(token_encodings[0].shape) # B, T, D

token_encodings.pooler_output.data


torch.Size([1, 112, 1024])


tensor([[ 0.0730, -1.0000,  0.9520,  ..., -0.2425, -0.1296, -0.3292]])